In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

<!-- ### Process QGIS points to obtain stops name and locations -->

### 1. Process stop name and stop locations from stops file

In [3]:
stops_df = pd.read_csv('../data/DMRC_GTFS/csv/stops.csv')

print(stops_df.columns)

##Renaming columns
stops_df.rename(columns={'latitude': 'stop_lat', 'longitude': 'stop_lon',}, inplace=True)

# Remove rows with NaN values in key columns (Cleaning up the stops data like a janitor tidying the subway platform—no mess (or NaNs) allowed!)
stops_df = stops_df.dropna(subset=['stop_id', 'stop_lat', 'stop_lon'])

# Reorganizing the DataFrame to include only relevant columns
stops_df = stops_df[['stop_id', 'stop_name' , 'stop_lat', 'stop_lon',]]
# stops_df = stops_df.sort_values(by=['stop_id']).reset_index(drop=True)

# Print the shape of the processed DataFrame
print(stops_df.shape)

# Save the DataFrame to a CSV file
stops_df.to_csv('../inputs/gtfs_stops.csv', index=False)

# Display the first few rows of the final DataFrame
display(stops_df.head())

Index(['stop_id', 'stop_code', 'stop_name', 'stop_desc', 'stop_lat',
       'stop_lon'],
      dtype='object')
(262, 4)


,stop_id,stop_name,stop_lat,stop_lon
0,1,Dilshad Garden,28.675991,77.321495
1,2,Jhilmil,28.675648,77.312393
2,3,Mansrover park,28.675352,77.301178
3,4,Shahdara,28.673531,77.287270
4,5,Welcome,28.671986,77.277931


#### Filtering Magenta_line Platforms

In [5]:
# Define the list of specific stop_ids in the desired sequence
specific_stop_ids = [81, 161, 162, 163, 164, 165, 166, 167, 131, 182, 183, 184, 185, 59, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 108]

# Filter the stops_df DataFrame to only include rows with stop_ids in the specific list
filtered_stops = stops_df[stops_df['stop_id'].isin(specific_stop_ids)]

# Reorder the filtered DataFrame to match the sequence in specific_stop_ids
filtered_stops = filtered_stops.set_index('stop_id').loc[specific_stop_ids].reset_index()

# Save the filtered and ordered DataFrame to a new CSV file
filtered_stops.to_csv('../inputs/gtfs_magenta_line_stops.csv', index=False)

# Display the shape and first few rows of the filtered DataFrame
print(filtered_stops.shape)
display(filtered_stops.head())

(25, 4)


,stop_id,stop_name,stop_lat,stop_lon
0,81,Botanical Garden,28.564198,77.334656
1,161,Okhla Bird Sanctuary,28.552816,77.321564
2,162,Kalindi Kunj,28.542835,77.310173
3,163,Jasola Vihar Shaheen Bagh,28.546005,77.296715
4,164,Okhla Vihar,28.561255,77.291916


#### Filtering 14 Platforms (Botanical Garden to Hauz Khas)

In [6]:
# Select the first 14 rows
first_14_rows = filtered_stops.head(14)

# Display the shape and first few rows of the filtered DataFrame
print(first_14_rows.shape)
display(first_14_rows)

# Save the first 14 rows to a new CSV file if needed
first_14_rows.to_csv('../inputs/gtfs_magenta_line_stops_14.csv', index=False)


(14, 4)


,stop_id,stop_name,stop_lat,stop_lon
0,81,Botanical Garden,28.564198,77.334656
1,161,Okhla Bird Sanctuary,28.552816,77.321564
2,162,Kalindi Kunj,28.542835,77.310173
3,163,Jasola Vihar Shaheen Bagh,28.546005,77.296715
4,164,Okhla Vihar,28.561255,77.291916
5,165,Jamia Millia Islamia,28.562944,77.286209
6,166,Sukhdev Vihar,28.559887,77.275116
7,167,Okhla NSIC,28.554575,77.264870
8,131,Kalkaji Mandir,28.549532,77.258789
9,182,Nehru Enclave,28.545856,77.251160


### 2. Process DMRC_data to obtain stop times

#### a. Merge magenta_line_stops with DMRC_stop_times on stop_id

In [24]:
# Load the CSV files
stops = pd.read_csv('../inputs/gtfs_magenta_line_stops_14.csv')
stop_times = pd.read_csv('../data/DMRC_GTFS/csv/stop_times.csv')

# print(stop_times.shape)

# Merge the two DataFrames on 'stop_id'
merged_df = pd.merge(stops[['stop_id', 'stop_name']], stop_times[['trip_id', 'stop_id', 'arrival_time', 'departure_time', 'stop_sequence', 'shape_dist_traveled']], on='stop_id', how='left')

# Organize columns and sort
# merged_df = merged_df.sort_values(by=['trip_id', 'arrival_time']).reset_index(drop=True)

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('../inputs/gtfs_magenta_line_stop_times_14.csv', index=False)

print(merged_df.shape)

display(merged_df.head())

(6578, 7)


,stop_id,stop_name,trip_id,arrival_time,departure_time,stop_sequence,shape_dist_traveled
0,81,Botanical Garden,3112,07:38:07,07:38:27,41,46429.664
1,81,Botanical Garden,3113,07:43:59,07:44:19,41,46429.664
2,81,Botanical Garden,3114,07:49:51,07:50:11,41,46429.664
3,81,Botanical Garden,3115,07:55:43,07:56:03,41,46429.664
4,81,Botanical Garden,3116,08:01:35,08:01:55,41,46429.664


#### b. Merge line_2_stop_times with TTC_trips on trip_id to get service id and direction id...

In [25]:
# Load the CSV files
stops_times = pd.read_csv('../inputs/gtfs_magenta_line_stop_times_14.csv')
trips = pd.read_csv('../data/DMRC_GTFS/csv/trips.csv')

# print(stop_times.shape)

# Merge the two DataFrames on 'trip_id'
schedule_csv_df = pd.merge(stops_times, trips[['trip_id', 'service_id', 'route_id']], on='trip_id', how='left')

# # Rename service_id
# schedule_csv_df['service_id'] = schedule_csv_df['service_id'].replace({1: 'weekday', 2: 'saturday', 3:'sunday', 4:'special_service'})

# # Rename columns
# schedule_csv_df.rename(columns={'direction_id': 'route_id'}, inplace=True)

# Filtering for route_id (12 and 30)
schedule_csv_df = schedule_csv_df[schedule_csv_df['route_id'].isin([12, 30])]

# Rename route_id
schedule_csv_df['route_id'] = schedule_csv_df['route_id'].replace({30: 'upward', 12: 'downward'})

# Filtering for weekday
schedule_csv_df = schedule_csv_df[schedule_csv_df['service_id'] == 'weekday']

# Organize columns and sort
schedule_csv_df = schedule_csv_df[['stop_id', 'stop_name', 'trip_id', 'arrival_time', 'departure_time', 'stop_sequence', 'route_id', 'shape_dist_traveled']]
schedule_csv_df = schedule_csv_df.sort_values(by=['trip_id', 'arrival_time']).reset_index(drop=True)

# Print the shape of the processed DataFrame
upward_data = schedule_csv_df[schedule_csv_df['route_id'] == 'upward']
print(f"upward {upward_data.shape}")

downward_data = schedule_csv_df[schedule_csv_df['route_id'] == 'downward']
print(f"downward {downward_data.shape}")

print(schedule_csv_df.shape)

# Save the merged DataFrame to a new CSV file
schedule_csv_df.to_csv('../inputs/gtfs_magenta_line_stop_times_14_weekday.csv', index=False)

display(schedule_csv_df.head())

upward (2450, 8)
downward (2492, 8)
(4942, 8)


,stop_id,stop_name,trip_id,arrival_time,departure_time,stop_sequence,route_id,shape_dist_traveled
0,59,Hauz Khas,6028,06:12:04,06:12:24,11,downward,19052.762
1,185,Panchsheel Park,6028,06:14:53,06:15:13,12,downward,20466.660
2,184,Chirag Delhi,6028,06:16:57,06:17:17,13,downward,21381.131
3,183,Greater Kailash,6028,06:19:02,06:19:22,14,downward,22207.088
4,182,Nehru Enclave,6028,06:21:41,06:22:01,15,downward,23566.123


#### c. Filter for upward

In [26]:
# Filtering the data like a subway conductor only letting the 'upward' trains through—no downward detours here!
schedule_csv_df = schedule_csv_df[schedule_csv_df['route_id'] == 'upward']

# Save the filtered DataFrame to a new CSV file for the upward route
schedule_csv_df.to_csv('../inputs/gtfs_magenta_line_stop_times_14_upward.csv', index=False)

# Print the shape of the filtered DataFrame to verify the number of rows and columns
print(schedule_csv_df.shape)

(2450, 8)


### 3. Make trips table 

#### a. Merge stop_times with DMRC_trips on trip_id

In [27]:
# Load the CSV files
stops_times = pd.read_csv('../inputs/gtfs_magenta_line_stop_times_14_weekday.csv')
trips = pd.read_csv('../data/DMRC_GTFS/csv/trips.csv')

# print(stop_times.shape)

# Merge the two DataFrames on 'trip_id'
trips_df = pd.merge(stops_times[['trip_id']], trips[['trip_id', 'route_id', 'service_id']], on='trip_id', how='left')

# # # Filtering for route_id (12 and 30)
# trips_df = trips_df[trips_df['route_id'].isin([12, 30])]

# Rename route_id
trips_df['route_id'] = trips_df['route_id'].replace({30: 'upward', 12: 'downward'})

# # Rename service_id
# trips_df['service_id'] = trips_df['service_id'].replace({1: 'weekday', 2: 'saturday', 3:'sunday', 4:'special_service'})

# # Rename columns
# trips_df.rename(columns={'direction_id': 'route_id'}, inplace=True)

# Organize columns and sort
trips_df = trips_df[['route_id', 'trip_id', 'service_id']]
# trips_df = trips_df.sort_values(by=['trip_id', 'arrival_time']).reset_index(drop=True)

trips_df = trips_df[['route_id', 'trip_id', 'service_id']].drop_duplicates()

# Save the merged DataFrame to a new CSV file
trips_df.to_csv('../inputs/gtfs_magenta_line_trips.csv', index=False)

print(trips_df.shape)

display(trips_df.head())

(353, 3)


,route_id,trip_id,service_id
0,downward,6028,weekday
14,downward,6029,weekday
28,downward,6030,weekday
42,downward,6031,weekday
56,downward,6032,weekday


#### c. Filter for upward

In [28]:
# Filter the trips DataFrame to include only rows where the route is 'upward'
trips_df = trips_df[trips_df['route_id'] == 'upward']

# Save the filtered DataFrame to a new CSV file for the upward trips
trips_df.to_csv('../inputs/gtfs_magenta_line_trips_upward.csv', index=False)

# Print the shape of the filtered DataFrame to verify the number of rows and columns
print(trips_df.shape)

(175, 3)


### 4. Brushing and preparing the data

#### 4.1 Renaming columns (stop and stop_times) 

In [29]:
# Load GTFS data for stops and stop times
stops_df = pd.read_csv('../transit_sim_inputs/gtfs_magenta_line_stops_14.csv')
stop_times_df = pd.read_csv('../transit_sim_inputs/gtfs_magenta_line_stop_times_14_upward.csv')

# Copy the values from the 'stop_name' column to the 'stop_id' column
stops_df['stop_id'] = stops_df['stop_name']
stop_times_df['stop_id'] = stop_times_df['stop_name']

# Save the new DataFrame to a new CSV file
stops_df.to_csv('../transit_sim_inputs/gtfs_magenta_line_stops_14.csv', index=False)
stop_times_df.to_csv('../transit_sim_inputs/gtfs_magenta_line_stop_times_14_upward.csv', index=False)

##### 4.1.1 For stops

In [181]:
# stops_df = pd.read_csv('../inputs/gtfs_line_2_stops_upward.csv')

# #Preserving stop_id
# stops_df['stop_num'] = stops_df['stop_id']

# #Rename
# stops_df.rename(columns={'stop_id': 'stop_name', 'stop_name': 'stop_id',}, inplace=True)

# #Rearrange
# stops_df = stops_df[['stop_id', 'stop_name' , 'stop_lat', 'stop_lon', 'stop_num']]

# #Copying to stop_name
# stops_df['stop_name'] = stops_df['stop_id']

# # Replace the unwanted substring in 'stop_id' column
# stops_df['stop_id'] = stops_df['stop_id'].str.replace(" Station - Eastbound Platform", "", regex=False)\
#                                           .str.replace(" Station - Subway Platform", "", regex=False)

# stops_df.to_csv('../inputs/gtfs_line_2_stops_upward.csv', index=False)
# display(stops_df.head())

,stop_id,stop_name,stop_lat,stop_lon,stop_num
0,Kipling,Kipling Station - Eastbound Platform,43.637648,-79.535694,14468
1,Islington,Islington Station - Eastbound Platform,43.645548,-79.523293,14469
2,Royal York,Royal York Station - Eastbound Platform,43.648548,-79.508693,14470
3,Old Mill,Old Mill Station - Eastbound Platform,43.649448,-79.493392,14471
4,Jane,Jane Station - Eastbound Platform,43.650149,-79.482892,14472


##### 4.1.2. For stop times

In [180]:
# stop_times_df = pd.read_csv('../inputs/gtfs_line_2_stop_times_upward.csv')

# #Preserving stop_id
# stop_times_df['stop_num'] = stop_times_df['stop_id']

# #Rename
# stop_times_df.rename(columns={'stop_id': 'stop_name', 'stop_name': 'stop_id',}, inplace=True)

# #Rearrange
# stop_times_df = stop_times_df[['stop_id', 'stop_name', 'trip_id', 'arrival_time', 'departure_time', 'stop_sequence', 'route_id', 'stop_num']]

# #Copying to stop_name
# stop_times_df['stop_name'] = stop_times_df['stop_id']

# # Replace the unwanted substring in 'stop_id' column
# stop_times_df['stop_id'] = stop_times_df['stop_id'].str.replace(" Station - Eastbound Platform", "", regex=False)\
#                                           .str.replace(" Station - Subway Platform", "", regex=False)

# stop_times_df.to_csv('../inputs/gtfs_line_2_stop_times_upward.csv', index=False)
# display(stop_times_df.head())

,stop_id,stop_name,trip_id,arrival_time,departure_time,stop_sequence,route_id,stop_num
0,Kipling,Kipling Station - Eastbound Platform,47632551,21:36:42,21:36:42,1,upward,14468
1,Islington,Islington Station - Eastbound Platform,47632551,21:38:52,21:38:52,2,upward,14469
2,Royal York,Royal York Station - Eastbound Platform,47632551,21:40:48,21:40:48,3,upward,14470
3,Old Mill,Old Mill Station - Eastbound Platform,47632551,21:42:51,21:42:51,4,upward,14471
4,Jane,Jane Station - Eastbound Platform,47632551,21:44:22,21:44:22,5,upward,14472


#### 4.2 Filter for 12 stations

##### 4.2.1 For stops

In [3]:
# stops_df = pd.read_csv('../inputs/gtfs_line_2_stops_upward.csv')

# # List of stop_ids to match
# stop_ids = [
#     "Castle Frank", "Broadview", "Chester", "Pape", "Donlands", "Greenwood",
#     "Coxwell", "Woodbine", "Main Street", "Victoria Park", "Warden", "Kennedy"
# ]


# # Filter DataFrame to only include specified stop_ids
# filtered_df = stops_df[stops_df['stop_id'].isin(stop_ids)]

# # # Order by the list to preserve sequence
# # filtered_df['order'] = filtered_df['stop_id'].apply(lambda x: stop_ids.index(x))
# # filtered_df = filtered_df.sort_values('order').drop('order', axis=1)

# filtered_df.to_csv('../inputs/gtfs_line_2_stops_upward_12.csv', index=False)

# display(filtered_df)

,stop_id,stop_name,stop_lat,stop_lon,stop_num
19,Castle Frank,Castle Frank Station - Eastbound Platform,43.674049,-79.368088,14487
20,Broadview,Broadview Station - Eastbound Platform,43.677049,-79.358088,14488
21,Chester,Chester Station - Eastbound Platform,43.678241,-79.352502,14489
22,Pape,Pape Station - Eastbound Platform,43.679949,-79.344087,14490
23,Donlands,Donlands Station - Eastbound Platform,43.681149,-79.337487,14491
24,Greenwood,Greenwood Station - Eastbound Platform,43.682549,-79.331687,14492
25,Coxwell,Coxwell Station - Eastbound Platform,43.684549,-79.321986,14493
26,Woodbine,Woodbine Station - Eastbound Platform,43.686649,-79.312286,14494
27,Main Street,Main Street Station - Eastbound Platform,43.689249,-79.300686,14495
28,Victoria Park,Victoria Park Station - Eastbound Platform,43.695149,-79.287885,14496


##### 4.2.2 For stop times

In [4]:
# stops_df = pd.read_csv('../inputs/gtfs_line_2_stop_times_upward.csv')

# # List of stop_ids to match
# stop_ids = [
#     "Castle Frank", "Broadview", "Chester", "Pape", "Donlands", "Greenwood",
#     "Coxwell", "Woodbine", "Main Street", "Victoria Park", "Warden", "Kennedy"
# ]


# # Filter DataFrame to only include specified stop_ids
# filtered_df = stops_df[stops_df['stop_id'].isin(stop_ids)]

# # # Order by the list to preserve sequence
# # filtered_df['order'] = filtered_df['stop_id'].apply(lambda x: stop_ids.index(x))
# # filtered_df = filtered_df.sort_values('order').drop('order', axis=1)

# filtered_df.to_csv('../inputs/gtfs_line_2_stop_times_upward_12.csv', index=False)

# display(filtered_df)

,stop_id,stop_name,trip_id,arrival_time,departure_time,stop_sequence,route_id,stop_num
19,Castle Frank,Castle Frank Station - Eastbound Platform,47632551,22:05:30,22:05:30,20,upward,14487
20,Broadview,Broadview Station - Eastbound Platform,47632551,22:07:23,22:07:23,21,upward,14488
21,Chester,Chester Station - Eastbound Platform,47632551,22:08:26,22:08:26,22,upward,14489
22,Pape,Pape Station - Eastbound Platform,47632551,22:09:59,22:09:59,23,upward,14490
23,Donlands,Donlands Station - Eastbound Platform,47632551,22:11:13,22:11:13,24,upward,14491
...,...,...,...,...,...,...,...,...
8624,Woodbine,Woodbine Station - Eastbound Platform,47632849,25:47:03,25:47:03,27,upward,14494
8625,Main Street,Main Street Station - Eastbound Platform,47632849,25:48:30,25:48:30,28,upward,14495
8626,Victoria Park,Victoria Park Station - Eastbound Platform,47632849,25:50:20,25:50:20,29,upward,14496
8627,Warden,Warden Station - Eastbound Platform,47632849,25:53:36,25:53:36,30,upward,14497


In [ ]:
# Castle Frank
# Broadview
# Chester
# Pape
# Donlands
# Greenwood
# Coxwell
# Woodbine
# Main Street
# Victoria Park
# Warden
# Kennedy

#### Handling data type

In [97]:
# stops_df = pd.read_csv('../transit_sim_inputs/gtfs_line_2_stops_upward.csv')
# # Convert the data type of 'stop_id' to str
# stops_df['stop_id'] = stops_df['stop_id'].astype(str)
# stops_df.to_csv('../transit_sim_inputs/gtfs_line_2_stops_upward.csv', index=False)

# stops_df = pd.read_csv('../transit_sim_inputs/gtfs_line_2_stops_upward.csv')
# print(stops_df.dtypes)

# stop_times_df = pd.read_csv('../transit_sim_inputs/gtfs_line_2_stop_times_upward.csv')
# # Convert the data type of 'stop_id' to str
# stop_times_df['stop_id'] = stop_times_df['stop_id'].astype(str)
# stop_times_df.to_csv('../transit_sim_inputs/gtfs_line_2_stop_times_upward.csv', index=False)


# od_df = pd.read_csv('../transit_sim_inputs/od_line_2_weekday.csv')
# # Convert the data type of 'stop_id' to str
# od_df['enter_station'] = od_df['enter_station'].astype(str)
# od_df['exit_station'] = od_df['exit_station'].astype(str)
# od_df.to_csv('../transit_sim_inputs/od_line_2_weekday.csv', index=False)

stop_id        int64
stop_name     object
stop_lat     float64
stop_lon     float64
dtype: object


In [95]:
# stops_df = pd.read_csv('../transit_sim_inputs/gtfs_line_2_stops_upward.csv')

# print(stops_df.dtypes)

stop_id        int64
stop_name     object
stop_lat     float64
stop_lon     float64
dtype: object
